## exercicio 1

In [1]:
import numpy as np
conec = np.array([[1,2],[2,3],[3,4],[4,5],[5,2],[5,3],[5,1]])
C= np.array ([2,2,1,2,1,2,2])

In [2]:
#

def Assembly(conec,C):
    nv = max(conec[:,0])#numero de nos 
    nc = len(conec[:,0])# numero de linhas

    A = np.zeros((nv,nv))
    for i in range(nc):
        n1=conec [i,0]-1
        n2=conec [i,1]-1

        A[n1,n2] += -C[i]
        A[n2,n1] += -C[i]
        A[n1,n1] += C[i]
        A[n2,n2] += C[i]

    return A


In [3]:
print('A = \n',Assembly(conec,C))

A = 
 [[ 4. -2.  0.  0. -2.]
 [-2.  5. -2.  0. -1.]
 [ 0. -2.  5. -1. -2.]
 [ 0.  0. -1.  3. -2.]
 [-2. -1. -2. -2.  7.]]


## exercicios 2

In [4]:
def SolveNetwork(conec, natm, nB, QB):
    Atilde = Assembly (conec, C)
    Atilde[natm,:] = 0

    for i in range(nB):
        Atilde[natm,i] = QB[i]
        
    
    
    b=np.zerps(natm)

    pressure = np.linalg.solve(Atilde,b)

    return pressure
    